In [32]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [33]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks



# Обработка естественного языка с нуля: перевод с помощью сети последовательностей и внимания
**Автор**: [Sean Robertson](https://github.com/spro)

В этом проекте будет происходить обучение нейронной сети переводу c русского на английский.

```sh
[KEY: > input, = target, < output]

> вы мне не поверите
= you re not going to believe me
< you re not going to believe me

> она всеми любима
= she is loved by everybody
< she is loved by all this family

> мы оба из австралии
= we re both from australia
< we re both from australia

> я герои тома
= i m tom s hero
< tom is tom s hero
```
... с разной степенью успеха.

Это стало возможным благодаря простой, но мощной идее сети последовательностей, в которой две рекуррентные нейронные сети работают вместе для преобразования одной последовательности в другую. Сеть кодировщика преобразует входную последовательность в вектор, а сеть декодера разворачивает этот вектор в новую последовательность.

![seq2seq](https://pytorch.org/tutorials/_images/seq2seq.png)

Чтобы улучшить эту модель, мы будем использовать [механизм](https://arxiv.org/abs/1409.0473), который позволяет декодеру научиться фокусироваться на определенном диапазоне входной последовательности.


In [34]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Загрузка файлов данных

Данные для этого проекта представляют собой набор из многих тысяч пар перевода с английского на русский язык.

Пары с английского на русский слишком велики, чтобы их можно было включить в репозиторий, поэтому, поэтому скачайте `data/rus.txt` перед продолжением. Файл представляет собой список пар перевода, разделенный табуляцией:

```sh
Go now.	А теперь уходи.
```
<div class="alert alert-info"><h4>Примечание</h4><p>Скачайте данные с
   <a href="https://www.manythings.org/anki/rus-eng.zip">сайта</a>
   и извлеките содержимое в текущую директорию.</p></div>



Подобно кодировке символов, используемой в учебных пособиях по RNN на уровне символов, мы будем представлять каждое слово в языке как вектор с единицей измерения или гигантский вектор нулей, за исключением одного (в индексе слова). По сравнению с десятками символов, которые могут существовать в языке, слов гораздо больше, поэтому вектор кодирования намного больше. Однако мы немного схитрим и обрежем данные, чтобы использовать только несколько тысяч слов на каждый язык.

![word encoding](https://pytorch.org/tutorials/_images/word-encoding.png)

Нам понадобится уникальный индекс для каждого слова, чтобы позже использовать его в качестве входных данных и целей сетей. Чтобы отслеживать все это, мы будем использовать вспомогательный класс под названием Lang, который имеет словари слово → индекс (`word2index`) и индекс → слово (`index2word`), а также количество каждого слова `word2count`, которое позже будет использоваться для замены редких слов.

In [35]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Все файлы представлены в формате Unicode, поэтому для упрощения мы преобразуем символы Unicode в ASCII, сделаем все строчными и урежем большую часть знаков препинания.

In [36]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^а-яА-Яa-zA-Z!?]+", r" ", s)
    return s.strip()

Чтобы прочитать файл данных, мы разделим файл на строки, а затем разобьем строки на пары. Все файлы — английский → другой язык, поэтому, если мы хотим перевести с другого языка → английский, я добавил флаг `reverse`, чтобы поменять местами пары.

In [37]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Поскольку примеров предложений *много* и мы хотим что-то быстро обучить, мы сократим набор данных до относительно коротких и простых предложений. Здесь максимальная длина составляет 10 слов (включая конечные знаки препинания), и мы фильтруем предложения, которые переводятся в форму «Я есть» или «Он есть» и т. д. (с учетом замененных ранее апострофов).

In [38]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

Полный процесс подготовки данных:

- Прочитать текстовый файл и разбить его на строки, разбивать строки на пары;
- Нормализовать текст, отфильтровать по длине и содержанию;
- Составить списки слов из предложений в парах.

In [39]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 487600 sentence pairs
Trimmed to 30189 sentence pairs
Counting words...
Counted words:
rus 10444
eng 4333
['он на два года старше меня', 'he s two years older than me']


## Модель Seq2Seq

Рекуррентная нейронная сеть, или RNN, — это сеть, которая работает с последовательностью и использует свои собственные выходные данные в качестве входных данных для последующих шагов.

[Сеть последовательного преобразования](https://arxiv.org/abs/1409.3215), или сеть seq2seq, или [сеть кодировщика-декодера](https://arxiv.org/pdf/1406.1078v3.pdf), — это модель, состоящая из двух RNN, называемых кодером и декодером. Кодер считывает входную последовательность и выводит один вектор, а декодер считывает этот вектор для создания выходной последовательности.

![seq2seq](https://pytorch.org/tutorials/_images/seq2seq.png)

В отличие от прогнозирования последовательности с помощью одной RNN, где каждый вход соответствует выходу, модель seq2seq освобождает нас от длины и порядка последовательности, что делает ее идеальной для перевода между двумя языками.

С помощью модели seq2seq кодер создает один вектор, который в идеальном случае кодирует «значение» входной последовательности в один вектор — одну точку в некотором N-мерном пространстве предложений.

### Кодер

Кодер сети seq2seq — это RNN, которая выводит некоторое значение для каждого слова из входного предложения. Для каждого входного слова кодер выводит вектор и скрытое состояние и использует скрытое состояние для следующего входного слова.

![seq2seq](https://pytorch.org/tutorials/_images/encoder-network.png)

In [40]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

### Декодер

Декодер — это еще одна RNN, которая принимает выходные векторы кодера и выводит последовательность слов для создания перевода.

#### Простой декодер

В простейшем декодере seq2seq мы используем только последний вывод кодера. Этот последний результат иногда называют *вектором контекста*, поскольку он кодирует контекст всей последовательности. Этот вектор контекста используется как начальное скрытое состояние декодера.

На каждом этапе декодирования декодеру предоставляется входной токен и скрытое состояние. Начальным входным токеном является токен начала строки `<SOS>`, а первым скрытым состоянием является вектор контекста (последнее скрытое состояние кодировщика).

![seq2seq](https://pytorch.org/tutorials/_images/decoder-network.png)


In [41]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

Я призываю вас тренироваться и наблюдать за результатами этой модели, но для экономии места мы сразу перейдем к золоту и введем механизм внимания.




#### Декодер внимания

Если между кодером и декодером передается только вектор контекста, этот единственный вектор несет бремя кодирования всего предложения.

Внимание позволяет сети декодера «сосредотачиваться» на различных частях выходных данных кодера на каждом этапе собственных выходных данных декодера. Сначала мы вычисляем набор *весов внимания*. Они будут умножены на выходные векторы кодера для создания взвешенной комбинации. Результат (называемый в коде `attn_applied`) должен содержать информацию об этой конкретной части входной последовательности и, таким образом, помогать декодеру выбирать правильные выходные слова.

![seq2seq](https://i.imgur.com/1152PYf.png)

Вычисление весов внимания выполняется с помощью другого слоя прямой связи «attn», используя в качестве входных данных входные данные декодера и скрытое состояние. Поскольку в обучающих данных присутствуют предложения всех размеров, для фактического создания и обучения этого слоя нам нужно выбрать максимальную длину предложения (входная длина для выходных данных кодера), к которой оно может применяться. В предложениях максимальной длины будут использоваться все веса внимания, в то время как в более коротких предложениях будут использоваться только первые несколько.

![seq2seq](https://pytorch.org/tutorials/_images/attention-decoder-network.png)

Внимание Багданау, также известное как аддитивное внимание, является широко используемым механизмом внимания в моделях последовательного перевода, особенно в задачах нейронного машинного перевода. Он был введен Богданау и др. в своей статье под названием «Нейронный машинный перевод путем совместного обучения выравниванию и переводу» (https://arxiv.org/pdf/1409.0473.pdf). Этот механизм внимания использует изученную модель выравнивания для вычисления оценок внимания между скрытыми состояниями кодера и декодера. Он использует нейронную сеть прямого распространения для расчета показателей выравнивания.

Однако существуют альтернативные механизмы внимания, такие как внимание Луонга, которое вычисляет оценки внимания, взяв скалярное произведение между скрытым состоянием декодера и скрытыми состояниями кодера. Здесь не используется нелинейное преобразование, используемое во внимании Богданау.

В этом проекте мы будем использовать внимание Богданау. Однако было бы ценным упражнением изучить возможность изменения механизма внимания для использования внимания Луонга.

In [42]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

<div class="alert alert-info"><h4>Примечание</h4><p>Существуют и другие формы внимания, которые позволяют обойти ограничение длины, используя подход относительного положения. Прочтите о «локальном внимании» в разделе <a href="https://arxiv.org/abs/1508.04025">Эффективные подходы к нейронному машинному переводу, основанному на внимании</a>.</p></div>

## Обучение

### Подготовка данных обучения

Для обучения для каждой пары нам понадобится входной тензор (индексы слов во входном предложении) и целевой тензор (индексы слов в целевом предложении). При создании этих векторов мы добавим токен EOS к обеим последовательностям.



In [43]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'rus', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

### Обучение модели

Для обучения мы пропускаем входное предложение через кодировщик и отслеживаем каждый вывод и последнее скрытое состояние. Затем декодер получает токен `<SOS>` в качестве первого входного сигнала и последнее скрытое состояние кодера в качестве его первого скрытого состояния.

«Принуждение учителя» — это концепция использования реальных целевых выходных данных в качестве каждого следующего входного сигнала вместо использования предположения декодера в качестве следующего входного сигнала. Использование принуждения учителя приводит к более быстрой сходимости, но [когда обученная сеть используется, она может проявлять нестабильность](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf).

Вы можете наблюдать выходные данные сетей, навязанных учителем, которые читают с последовательной грамматикой, но отклоняются далеко от правильного перевода - интуитивно они научились представлять выходную грамматику и могут «уловить» смысл, как только учитель скажет им первые несколько слов, но он вообще не научился должным образом создавать предложение на основе перевода.

Благодаря свободе, которую дает нам автоградация PyTorch, мы можем случайным образом выбирать, использовать принуждение учителя или нет с помощью простого оператора if. Включите `teacher_forcing_ratio`, чтобы использовать его больше.

In [44]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

Это вспомогательная функция для печати прошедшего времени и расчетного оставшегося времени с учетом текущего времени и процента прогресса.

In [45]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Весь процесс обучения выглядит так:

- Запустить таймер;
- Проинициализировать оптимизаторы и критерии;
- Создать набор обучающих пар;
- Запустить пустой массив потерь для построения графика.

Затем мы много раз вызываем `train` и время от времени печатаем прогресс (%
примеров, время на данный момент, расчетное время) и средние потери.

In [46]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

### Отображение результатов

Построение графика выполняется с помощью matplotlib с использованием массива значений потерь `plot_losses`, сохраненного во время обучения.




In [47]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Оценка

Оценка в основном аналогична обучению, но здесь нет целей, поэтому мы просто передаем прогнозы декодера самому себе для каждого шага. Каждый раз, когда он предсказывает слово, мы добавляем его в выходную строку, и если он предсказывает токен EOS, мы на этом останавливаемся. Мы также сохраняем выходные данные внимания декодера для последующего отображения.




In [48]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

Мы можем оценить случайные предложения из обучающего набора и распечатать входные, целевые и выходные данные, чтобы сделать некоторые субъективные оценки качества:

In [49]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Обучение и оценка

Имея все эти вспомогательные функции (это выглядит как дополнительная работа, но упрощает проведение нескольких экспериментов), мы фактически можем инициализировать сеть и начать обучение.

Помните, что входные предложения были тщательно отфильтрованы. Для этого небольшого набора данных мы можем использовать относительно небольшие сети из 256 скрытых узлов и одного уровня GRU.

<div class="alert alert-info">
  <h4>Примечание</h4>
  <p>Если вы запустите этот блокнот, вы сможете тренироваться, прерывать работу ядра, оценивать и продолжать позже. Закомментируйте строки, в которых инициализируются кодировщик и декодер, и снова запустите <code>trainIters</code>.
  </p>
</div>




In [50]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading lines...
Read 487600 sentence pairs
Trimmed to 30189 sentence pairs
Counting words...
Counted words:
rus 10444
eng 4333
10m 8s (- 152m 4s) (5 6%) 1.2638
20m 9s (- 141m 8s) (10 12%) 0.5043
30m 0s (- 130m 2s) (15 18%) 0.2847
39m 46s (- 119m 18s) (20 25%) 0.1883
49m 39s (- 109m 14s) (25 31%) 0.1408
59m 32s (- 99m 14s) (30 37%) 0.1153
69m 21s (- 89m 10s) (35 43%) 0.0991
79m 12s (- 79m 12s) (40 50%) 0.0882
89m 26s (- 69m 34s) (45 56%) 0.0811
99m 25s (- 59m 39s) (50 62%) 0.0754
109m 14s (- 49m 39s) (55 68%) 0.0713
119m 9s (- 39m 43s) (60 75%) 0.0682
129m 10s (- 29m 48s) (65 81%) 0.0659
139m 9s (- 19m 52s) (70 87%) 0.0634
148m 57s (- 9m 55s) (75 93%) 0.0614
158m 36s (- 0m 0s) (80 100%) 0.0602


Установим выпадающие слои в режим `eval`.

In [51]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> вы мне не поверите
= you re not going to believe me
< you re not going to believe me <EOS>

> она всеми любима
= she is loved by everybody
< she is loved by all this family <EOS>

> мы оба из австралии
= we re both from australia
< we re both from australia <EOS>

> я герои тома
= i m tom s hero
< tom is tom s hero <EOS>

> я ничему не учусь
= i m not learning anything
< i m not learning anything <EOS>

> вы сильнее нас
= you re stronger than us
< you re stronger than us <EOS>

> сеичас у меня нет недомогания
= i m not feeling bad at the moment
< i m not feeling now <EOS>

> я оптимист
= i m an optimist
< i m optimistic <EOS>

> я пишу статью для школьнои газеты
= i m writing an article for the school newspaper
< i m the only one who old the day off

> я второкурсник
= i m a sophomore
< she is a sophomore <EOS>



### Визуализация внимания

Полезным свойством механизма внимания является то, что его результаты легко интерпретируются. Поскольку он используется для взвешивания конкретных выходных данных кодера входной последовательности, мы можем представить, как смотрим, где сеть сосредоточена больше всего на каждом временном шаге.

Вы можете просто запустить `plt.matshow(attentions)`, чтобы увидеть вывод внимания, отображаемый в виде матрицы. Для лучшего просмотра мы проделаем дополнительную работу по добавлению осей и меток:

In [83]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])

evaluateAndShowAttention('это скучная книга')
evaluateAndShowAttention('мы в безопасности')
evaluateAndShowAttention('том в тюрьме')
evaluateAndShowAttention('мы можем вам помочь')
evaluateAndShowAttention('придется подождать')

input = это скучная книга
output = i m eating this with this <EOS>


<ipython-input-83-b32e3d7a2ebd>:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
<ipython-input-83-b32e3d7a2ebd>:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)


input = мы в безопасности
output = we re safe in the safe <EOS>
input = том в тюрьме
output = tom s tom in jail <EOS>
input = мы можем вам помочь
output = we re able to help you <EOS>
input = придется подождать
output = you re going to have kept wait welcome <EOS>


## Выводы

Как видно из результатов, перевод оставляет желать лучшего. Также наличие того факта, что при переводе какого-либо слова, которое отсутствует во внутреннем словаре обучающего датасета, выбрасывается исключение, оставляет весьма негативные впечатлении о качестве такого переводчика. Однако, как минимальная интерпретация работы переводчика «изнутри», данный код дает хорошее понимание работы переводчиков в целом.


